In [1]:
import librosa
import numpy as np
from scipy.signal import butter,freqz, findfreqs,filtfilt
from scipy import signal
import os
import pandas as pd

In [2]:
# Tiền xử lý dữ liệu âm thanh
fs = 44100
lowcut = 20
highcut = 20000
Wn = (2 * lowcut / fs, 2 * highcut / fs) 

def preprocess_audio(audio_file):
    # Chuyển đổi tần số lấy mẫu
    signal, sr = librosa.load(audio_file, sr=22050)
    # Cắt 1 giây đầu của tệp âm thanh
    signal = signal[sr:]
    # Lọc tạp âm bằng bộ lọc số học
    b, a = butter(2, Wn, btype='band')
    signal_filtered = filtfilt(b, a, signal)
    return signal_filtered, sr


In [3]:
# Trích xuất đặc trưng MFCC
def extract_features(signal, sr):
    # Sử dụng khung cửa sổ trượt độ dài 512 mẫu
    hop_length = 512
    # Trích xuất đặc trưng MFCC với số cấp MFCC là 13
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13, hop_length=hop_length)
    # Trung bình các giá trị MFCC trên toàn bộ tệp âm thanh
    features = np.mean(mfccs.T, axis=0)
    return features

In [4]:
from sklearn.ensemble import RandomForestClassifier
def train_model(X_train, y_train):
    rf_model = RandomForestClassifier(n_estimators=100)
    rf_model.fit(X_train, y_train)
    return rf_model


In [5]:
def predict_audio(audio_file, model):
    signal, sr = preprocess_audio(audio_file)
    features = extract_features(signal, sr)
    prediction = model.predict([features])
    return prediction[0]

In [6]:
# Đọc dữ liệu từ file CSV
data = pd.read_csv('data.csv')
print(data)

                     File                           label
0                mbn1.wav                    máy bơm nước
1                mbn2.wav                    máy bơm nước
2                mbn3.wav                    máy bơm nước
3                mbn4.wav                    máy bơm nước
4                mbn5.wav                    máy bơm nước
..                    ...                             ...
132  fruit_juicer (5).mp3  máy ép trái cây (fruit juicer)
133  fruit_juicer (6).mp3  máy ép trái cây (fruit juicer)
134  fruit_juicer (7).mp3  máy ép trái cây (fruit juicer)
135  fruit_juicer (8).mp3  máy ép trái cây (fruit juicer)
136  fruit_juicer (9).mp3  máy ép trái cây (fruit juicer)

[137 rows x 2 columns]


In [7]:
# Khởi tạo dictionary để lưu trữ đường dẫn đến các tệp âm thanh
audio_files = {}

# Lặp qua từng hàng trong dữ liệu
for index, row in data.iterrows():
    # Lấy đường dẫn và nhãn từ các cột tương ứng
    file_path = row['File']
    label = row['label']
    
    # Thêm đường dẫn và nhãn vào dictionary
    audio_files[file_path] = label
print(audio_files)

{'mbn1.wav': 'máy bơm nước', 'mbn2.wav': 'máy bơm nước', 'mbn3.wav': 'máy bơm nước', 'mbn4.wav': 'máy bơm nước', 'mbn5.wav': 'máy bơm nước', 'mbn6.wav': 'máy bơm nước', 'mbn7.wav': 'máy bơm nước', 'mbn8.wav': 'máy bơm nước', 'mc1.wav': 'máy cưa', 'mc10.wav': 'máy cưa', 'mc11.wav': 'máy cưa', 'mc12.wav': 'máy cưa', 'mc2.wav': 'máy cưa', 'mc3.wav': 'máy cưa', 'mc4.wav': 'máy cưa', 'mc5.wav': 'máy cưa', 'mc6.wav': 'máy cưa', 'mc7.wav': 'máy cưa', 'mc8.wav': 'máy cưa', 'mc9.wav': 'máy cưa', 'lawn_tractor (1).mp3': 'máy cắt cỏ (lawn tractor)', 'lawn_tractor (10).mp3': 'máy cắt cỏ (lawn tractor)', 'lawn_tractor (11).mp3': 'máy cắt cỏ (lawn tractor)', 'lawn_tractor (12).mp3': 'máy cắt cỏ (lawn tractor)', 'lawn_tractor (13).mp3': 'máy cắt cỏ (lawn tractor)', 'lawn_tractor (14).mp3': 'máy cắt cỏ (lawn tractor)', 'lawn_tractor (15).mp3': 'máy cắt cỏ (lawn tractor)', 'lawn_tractor (2).mp3': 'máy cắt cỏ (lawn tractor)', 'lawn_tractor (3).mp3': 'máy cắt cỏ (lawn tractor)', 'lawn_tractor (4).mp3': '

In [8]:
features_list = []
labels_list = []
for file, label in audio_files.items():
    file_path = label+'/'+file
    signal, sr = preprocess_audio(file_path)
    features = extract_features(signal, sr)
    features_list.append(features)
    labels_list.append(label)

In [9]:
df = pd.DataFrame({'File': audio_files.keys(), 'mfcc': features_list})
mfcc_df = pd.DataFrame(df['mfcc'].tolist(), columns=['mfcc_{}'.format(i) for i in range(13)])
result_df = pd.concat([df['File'], mfcc_df], axis=1)
print(result_df)

if os.path.exists('result.csv'):
    os.remove('result.csv')
    
result_df.to_csv('result.csv', index=False)

                     File      mfcc_0      mfcc_1     mfcc_2     mfcc_3  \
0                mbn1.wav -268.443020   41.455886 -40.612264 -17.308881   
1                mbn2.wav -157.457301  101.974231 -42.588496  31.309613   
2                mbn3.wav   22.725151   55.852903 -27.948332  20.358179   
3                mbn4.wav -233.081943  192.381976 -19.426254  31.769795   
4                mbn5.wav  -36.186673  117.908198 -16.141211  15.004685   
..                    ...         ...         ...        ...        ...   
132  fruit_juicer (5).mp3  -71.217267   23.673646 -31.115233  35.601067   
133  fruit_juicer (6).mp3  -71.033822   24.022198 -31.086751  35.630850   
134  fruit_juicer (7).mp3  -71.145664   23.826301 -31.127248  35.639239   
135  fruit_juicer (8).mp3  -71.196352   23.486149 -31.001125  35.437178   
136  fruit_juicer (9).mp3  -70.823922   24.739848 -30.434238  36.081140   

        mfcc_4     mfcc_5     mfcc_6     mfcc_7     mfcc_8     mfcc_9  \
0   -31.906897 -17.820238 

In [10]:
# Huấn luyện mô hình
rf_model = train_model(features_list, labels_list)

In [11]:
import ipywidgets as widgets
from IPython.display import display

uploader = widgets.FileUpload(
    accept='*.csv',  
    multiple=False 
)

In [12]:
def handle_upload(change):
    uploaded_file = uploader.value
    if uploaded_file:
        file_name = next(iter(uploaded_file))
        file_content = uploaded_file[file_name]['content']
        
        current_directory = os.getcwd()
        file_path = os.path.join(current_directory, file_name)
        with open(file_path, 'wb') as file:
            file.write(file_content)

        prediction = predict_audio(file_path, rf_model)

        print(f'Phân loại của tệp âm thanh {file_path} là: {prediction}')

uploader.observe(handle_upload, names='value')
display(uploader)

FileUpload(value={}, accept='*.csv', description='Upload')